# Clustering Voxel data with k-means++ and 3D-CNN

## for Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q /content/drive/MyDrive/data/mn10_64.zip

In [ ]:
!pip install tensorflow-determinism kaleido

In [ ]:
import os
import datetime
import random
from glob import glob
import re
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


In [ ]:
def set_seed(seed=200):
    tf.random.set_seed(seed)

    # optional
    # for numpy.random
    np.random.seed(seed)
    # for built-in random
    random.seed(seed)
    # for hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    
set_seed(123)

In [ ]:
EXPERIMENT_DIR = '3DCNNx4_64_GAP_pca'
NUM_CLASSES = 10
NUM_CLUSTERS = 10
NUM_PCA_COMPONENTS = 10
BUFFER_SIZE = 1000
BATCH_SIZE = 8

CLUSTERING_INTERVAL = 5
WHOLE_CYCLES = 20
num_epochs = CLUSTERING_INTERVAL * WHOLE_CYCLES
print('total epochs: ', num_epochs)

DATA_DIR = '/content/mn10/64'
IMAGE_SIZE = 64
NUM_CHANNELS = 1

In [ ]:
os.makedirs(EXPERIMENT_DIR, exist_ok=True)

## Define models

In [ ]:
feature_extractor = keras.Sequential([
    layers.Conv3D(filters=64, kernel_size=3, padding='same', activation='relu',
                  input_shape=(IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)),
    layers.MaxPool3D(pool_size=2),
    layers.BatchNormalization(),

    layers.Conv3D(filters=64, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPool3D(pool_size=2),
    layers.BatchNormalization(),

    layers.Conv3D(filters=128, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPool3D(pool_size=2),
    layers.BatchNormalization(),

    layers.Conv3D(filters=128, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPool3D(pool_size=2),
    layers.BatchNormalization(),

    layers.GlobalAveragePooling3D(),
])

In [ ]:
feature_extractor.summary()

In [ ]:
classifier = keras.Sequential([
    feature_extractor,
    layers.Activation(keras.activations.relu),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
classifier.summary()

In [ ]:
tf.keras.utils.plot_model(feature_extractor,
                         to_file=os.path.join(EXPERIMENT_DIR, 'feature_extractor.png'),
                         show_shapes=True)

In [ ]:
tf.keras.utils.plot_model(classifier,
                         to_file=os.path.join(EXPERIMENT_DIR, 'classifier.png'),
                         show_shapes=True)

In [ ]:
pca = PCA(n_components=NUM_PCA_COMPONENTS)
stdsc = StandardScaler()
kmc = KMeans(n_clusters=NUM_CLUSTERS, init='k-means++', n_init=10, max_iter=300,
                       tol=0.0001, verbose=0, random_state=123, copy_x=True)

## Prepare Data

In [ ]:
categories = ['bathtub', 'bed', 'chair', 'desk', 'dresser',
              'monitor', 'night_stand', 'sofa', 'table', 'toilet']

In [ ]:
train_pattern = DATA_DIR +'/train/*.npy'

train_list_ds = tf.data.Dataset.list_files(train_pattern, shuffle=False)

In [ ]:
cat_re = re.compile(r'.+/(.+?)_[0-9]+\.npy')
train_labels = [cat_re.match(item.numpy().decode())[1] for item in train_list_ds]
train_ids = [categories.index(cat) for cat in train_labels]
train_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(train_ids, tf.int64))

In [ ]:
def read_npy_file(path):
    data = np.load(path.numpy())
    data = np.expand_dims(data, axis=-1)
    return tf.convert_to_tensor(data, dtype=tf.float32)

In [ ]:
train_3d_ds = train_list_ds.map(
        lambda item: tf.py_function(read_npy_file, [item], tf.float32)).cache(filename='./cache.tf-data')
train_dataset = tf.data.Dataset.zip((train_3d_ds, train_label_ds)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Train models

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = os.path.join(EXPERIMENT_DIR, 'logs', current_time , 'train')
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1.0e-4)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

for epoch in range(num_epochs):
    with train_summary_writer.as_default():
        if epoch % CLUSTERING_INTERVAL == 0:
            features_list = [feature_extractor(batch, training=False) for batch in train_3d_ds.batch(BATCH_SIZE)]
            features = np.vstack(features_list)
            features_pca = pca.fit_transform(features)
            features_std = stdsc.fit_transform(features_pca)
            km_predictions = kmc.fit_predict(features_std)
            cluster_matrix = np.zeros((NUM_CLASSES, NUM_CLUSTERS), dtype=np.int32)
            for i, cat_id in enumerate(train_ids):
                cluster_matrix[cat_id, km_predictions[i]] += 1
            print("Epoch: {}, Distortion: {:.2f}".format(epoch, kmc.inertia_))
            tf.summary.scalar('sse', kmc.inertia_, step=epoch)
            print(cluster_matrix)
            pseudo_label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(km_predictions, tf.int64))
            train_dataset = tf.data.Dataset.zip((train_3d_ds, pseudo_label_ds)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

            with tf.GradientTape() as tape:

                y = classifier(x_batch_train, training=True)

                loss = loss_fn(y_batch_train, y)

            grads = tape.gradient(loss, classifier.trainable_weights)

            optimizer.apply_gradients(zip(grads, classifier.trainable_weights))

            train_loss(loss)
            train_accuracy(y_batch_train, y)

        template = 'Epoch {}, Loss: {}, Accuracy: {}'
        print (template.format(epoch+1,
                             train_loss.result(),
                             train_accuracy.result()*100))

        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

        train_loss.reset_states()
        train_accuracy.reset_states()


### Save models

In [ ]:
feature_extractor.save(os.path.join(EXPERIMENT_DIR, 'saved_models', 'feature_extractor'))
classifier.save(os.path.join(EXPERIMENT_DIR, 'saved_models', 'classifier'))